<a href="https://colab.research.google.com/github/Ivan-Nebogatikov/HumanActivityRecognition/blob/master/Open_Dataset_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Create frames

In [36]:
import numpy as np
import pandas as pd
import json
from datetime import datetime
from datetime import date

def getValue(x):
    l = list(json.loads(x.replace('\'', '"')))
    return l

def getDiff(x):
    res = list()
    for i, v in enumerate(x):
        res.append(v)
        #if i > 0:
        #    res.append(v - x[i - 1])
        #else:
        #    res.append(0)
    return res

heart = pd.read_csv('https://raw.githubusercontent.com/Ivan-Nebogatikov/HumanActivityRecognition/master/datasets/2282_3888_bundle_archive/heart.csv')
heart['timestamp'] = heart['timestamp'].map(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f"))
heart = heart.sort_values(by='timestamp')

def getHeart(x):
    dt = datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f")
    f = heart[heart['timestamp'] < dt]
    lastValue = f.iloc[[-1]]['values'].tolist()[0]
    intValue = list(json.loads(lastValue.replace('\'', '"')))[0]
    return intValue

acc = pd.read_csv('https://raw.githubusercontent.com/Ivan-Nebogatikov/HumanActivityRecognition/master/datasets/2282_3888_bundle_archive/acc.csv')
acc['heart'] = acc['timestamp'].map(lambda x: getHeart(x))

def change(x):
    if x == 'Pause' or x == 'Movie':
        x = 'Watching TV'
    if x == 'Shop':
        x = 'Walk'
    if x == 'Football':
        x = 'Running'
    if x == 'Meeting' or x == 'Work' or x == 'Picnic ' or x == 'In vehicle' or x == 'In bus' :
        x = 'Sitting'
    if x == 'On bus stop':
        x = 'Walk'
    if x == 'Walking&party' or x == 'Shopping& wearing' or x == 'At home':
        x = 'Walk'
    return x
acc['act'] = acc['act'].map(lambda x: change(x))

labels = np.array(acc['act'])

arrays = acc['values'].map(lambda x: getValue(x))
x = getDiff(list(arrays.map(lambda x: np.double(x[0]))))
y = getDiff(list(arrays.map(lambda x: np.double(x[1]))))
z = getDiff(list(arrays.map(lambda x: np.double(x[2]))))


Define a function to predict activity with random forest

In [30]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

def Predict(x):
    train_features, test_features, train_labels, test_labels = train_test_split(x, labels, test_size = 0.15, random_state = 242)
    print('Training Features Shape:', train_features.shape)
    print('Testing Features Shape:', test_features.shape)
    print("\n")

    rf = RandomForestClassifier(n_estimators = 400, random_state = 3, class_weight='balanced') # Классификатор с 100 деревьями
    rf.fit(train_features, train_labels);

    predictions = list(rf.predict(test_features))
    pred_prob = rf.predict_proba(test_features)

    class_report(
        y_true=test_labels, 
        y_pred=np.asarray(predictions), 
        y_score=pred_prob, average='micro')

    print(rf.feature_importances_)

Predict with heart data

In [39]:
frame = pd.DataFrame(list(acc['heart']), columns=['h'])
feature_list = list(frame.columns)
print(frame)
Predict(frame)

         h
0     79.0
1     76.0
2     72.0
3     71.0
4     71.0
...    ...
4737  73.0
4738  70.0
4739  72.0
4740  73.0
4741  72.0

[4742 rows x 1 columns]
Training Features Shape: (4030, 1)
Testing Features Shape: (712, 1)


Accuracy: 0.14747191011235955
                                      precision    recall  ...   pred       AUC
Cooking                                0.171429  0.230769  ...   35.0  0.750000
Eat                                    0.189189  0.259259  ...   74.0  0.683665
In computer                            0.250000  0.093023  ...   48.0  0.565559
Phone was out of the pocket (forgot)   0.019868  0.500000  ...  151.0  0.691572
Running                                0.594595  0.578947  ...   37.0  0.879178
Sitting                                0.290323  0.065693  ...   31.0  0.593627
Sleep                                  0.265823  0.403846  ...   79.0  0.693415
Train                                  0.064935  0.217391  ...   77.0  0.639364
Video games            

Predict with accelerometer

In [31]:
frame = pd.DataFrame(zip(x, y, z), columns=['x', 'y', 'z'])
feature_list = list(frame.columns)
print(frame)
Predict(frame)

              x          y         z
0    -10.173126  -1.471237  1.924393
1      4.726227 -10.251968  0.491104
2    -10.776581  -0.819656 -0.581741
3    -10.300018  -1.300781  0.812164
4     -8.121658  -0.150238 -0.705032
...         ...        ...       ...
4737  -9.694046  -1.125046  2.037765
4738 -10.787354  -2.289215  1.636612
4739  -4.396042  -7.009506  2.160950
4740  -9.635635   1.458542 -0.267487
4741  -5.556152  -7.290588 -2.863174

[4742 rows x 3 columns]
Training Features Shape: (4030, 3)
Testing Features Shape: (712, 3)


Accuracy: 0.42837078651685395
                                      precision    recall  ...   pred       AUC
Cooking                                0.250000  0.115385  ...   12.0  0.792779
Eat                                    0.393939  0.240741  ...   33.0  0.780057
In computer                            0.457364  0.457364  ...  129.0  0.770287
Phone was out of the pocket (forgot)   0.000000  0.000000  ...    1.0  0.713645
Running                        

Predict with both sensors

In [40]:
frame = pd.DataFrame(zip(list(acc['heart']), x, y, z), columns=['h', 'x', 'y', 'z'])
feature_list = list(frame.columns)
print(frame)
Predict(frame)

         h          x          y         z
0     79.0 -10.173126  -1.471237  1.924393
1     76.0   4.726227 -10.251968  0.491104
2     72.0 -10.776581  -0.819656 -0.581741
3     71.0 -10.300018  -1.300781  0.812164
4     71.0  -8.121658  -0.150238 -0.705032
...    ...        ...        ...       ...
4737  73.0  -9.694046  -1.125046  2.037765
4738  70.0 -10.787354  -2.289215  1.636612
4739  72.0  -4.396042  -7.009506  2.160950
4740  73.0  -9.635635   1.458542 -0.267487
4741  72.0  -5.556152  -7.290588 -2.863174

[4742 rows x 4 columns]
Training Features Shape: (4030, 4)
Testing Features Shape: (712, 4)


Accuracy: 0.5014044943820225
             precision    recall  f1-score  support   pred       AUC
Cooking       0.476190  0.384615  0.425532     26.0   21.0  0.898716
Eat           0.407407  0.203704  0.271605     54.0   27.0  0.830646
In computer   0.533333  0.496124  0.514056    129.0  120.0  0.821785
Running       0.741935  0.605263  0.666667     38.0   31.0  0.436221
Sitting       0

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
